In [1]:
import pandas as pd
import pandas_ta as ta
import talib
from stock_env.feature.feature_extractor import *

# Data Loader

In [2]:
from stock_env.data_loader.vndirect_loader import VNDDataLoader

vndloader = VNDDataLoader(
    symbols=['SSI'],
    start='2021-01-01',
    end='2022-12-31'
)

pull_df = vndloader.download()
raw_data = pull_df[['date', 'adOpen', 'adHigh', 'adLow', 'adClose', 'volume', 'code']]
df = raw_data.rename(
    columns={
        'code': 'ticker',
        'date': 'time',
        'adOpen': 'open',
        'adHigh': 'high',
        'adLow': 'low',
        'adClose': 'close',
    }
)
df = df.sort_values(by='time').reset_index(drop=True)
temp_df = df.copy()

In [5]:
temp_df = df.copy()

In [10]:
feature_extractor = SimpleTrendFeatures().preprocess(df)

In [12]:
feature_extractor

,ADX_50,DMP_50,AROOND_50,AROONU_50,AROONOSC_50,STC_10_10_20_0.5,STCmacd_10_10_20_0.5,STCstoch_10_10_20_0.5,NATR_50,RSI_50,...,MACDh_20_50_10,MACDs_20_50_10,MACD_50_100_10,MACDh_50_100_10,MACDs_50_100_10,MACD_100_200_10,MACDh_100_200_10,MACDs_100_200_10,LOW_ratio,VOLUME_ratio
208,0.131335,0.271248,0.54,0.20,-0.34,0.000000,0.299720,0.000000,0.029138,0.567584,...,0.114910,0.273537,2.062367,-0.119145,2.181511,6.679371,-0.199708,6.879079,1.003948,1.753842
209,0.131567,0.268541,0.52,0.18,-0.34,0.000000,0.398042,0.000000,0.028901,0.565947,...,0.162457,0.309639,2.063501,-0.096554,2.160055,6.646909,-0.189957,6.836866,1.003948,1.000705
210,0.132724,0.284733,0.50,0.16,-0.34,0.000000,0.589041,0.000000,0.028226,0.585535,...,0.257203,0.366795,2.091020,-0.056484,2.147503,6.628334,-0.170618,6.798951,1.003948,1.995284
211,0.133857,0.280101,0.48,0.14,-0.34,0.000000,0.704646,0.000000,0.028237,0.582196,...,0.308320,0.435310,2.111817,-0.029197,2.141015,6.607228,-0.156865,6.764093,1.003948,1.016310
212,0.135482,0.288327,0.46,1.00,0.54,0.000000,0.809278,0.000000,0.028051,0.587016,...,0.351622,0.513448,2.136772,-0.003471,2.140243,6.589000,-0.143258,6.732257,1.003948,1.214544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,0.267840,0.226386,0.64,0.92,0.28,0.875000,0.908873,0.780335,0.055245,0.491737,...,0.542127,-0.224961,-1.690163,0.322271,-2.012434,-3.587530,0.145345,-3.732875,1.000000,0.985304
485,0.264622,0.219813,0.62,0.90,0.28,0.933220,0.923570,0.773654,0.052342,0.511769,...,0.528648,-0.107484,-1.610188,0.329111,-1.939298,-3.544914,0.153786,-3.698700,1.000000,0.944558
486,0.260964,0.227370,0.60,1.00,0.40,0.928486,0.892483,0.720836,0.052957,0.506877,...,0.491459,0.001729,-1.539444,0.327153,-1.866598,-3.505815,0.157815,-3.663630,1.157088,0.788792
487,0.257486,0.225639,0.58,0.98,0.40,0.829196,0.867003,0.569571,0.051908,0.509270,...,0.458696,0.103662,-1.468714,0.325542,-1.794255,-3.465703,0.161940,-3.627644,1.168582,0.617927
